In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/Capstone_Email_Dataset

/content/drive/MyDrive/Capstone_Email_Dataset


In [3]:
import os
# Get the current working directory
cwd = os.getcwd()
# Get the list of all subdirectories in the current working directory
subdirectories = [f for f in os.listdir(cwd) if os.path.isdir(f)]

# Data Pre-Processing

In [4]:
!pip install git+https://github.com/vTheWise/neattext.git

  Cloning https://github.com/vTheWise/neattext.git to /tmp/pip-req-build-7mzq7fke
  Running command git clone --filter=blob:none --quiet https://github.com/vTheWise/neattext.git /tmp/pip-req-build-7mzq7fke
  Resolved https://github.com/vTheWise/neattext.git to commit f13c91ab7f6fea505bdc74d335bf8d2a14008728
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neattext: filename=neattext-0.1.4-py3-none-any.whl size=114864 sha256=d8cb7d1f1e79c7c76b40afa5d3e7de8f8a83412a6d11dbb0bc9b683e60e2f760
  Stored in directory: /tmp/pip-ephem-wheel-cache-lecu1p25/wheels/c0/6d/08/74495672891de18d9e5c8e1db78dfdd7451929d5dfb9fc4de5
Successfully built neattext


In [4]:
# !pip install neattext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.6 MB/s eta 0:00:00


In [7]:
!pip install striprtf

In [5]:
import neattext as nt
from neattext.functions import clean_text

tf = []

for sub in subdirectories:
  files = os.listdir(sub)
  if "benign" in sub:
    label = 0
  else:
    label = 1
  for file in files:
      name, ext = os.path.splitext(file)
      if ext == ".txt":
        f = open(os.path.join(sub, file), "rb")
        content = f.read()
        ct = clean_text(content.decode(), puncts=True, urls=True, emails=True, numbers=True, phone_num=True, multiple_whitespaces=False)
        tf.append([label, nt.TextFrame(text=ct)])
        f.close()

In [12]:
tf_lst_of_dict = []

for t in tf:
  dict_t = t[1].describe()
  if dict_t:
    dict_t['score_readability'] = round(t[1].readability()['automated readability'], 2)
    dict_t['label'] = t[0]
    tf_lst_of_dict.append(dict_t)

In [13]:
tf_lst_of_dict[561]

{'Length': 287,
 'vowels': 90,
 'consonants': 160,
 'stopwords': 0,
 'punctuations': 1,
 'special_char': 1,
 'tokens(whitespace)': 37,
 'tokens(words)': 37,
 'score_readability': 33.6,
 'label': 1}

In [14]:
import pandas as pd

df = pd.DataFrame.from_dict(tf_lst_of_dict)

In [15]:
df.head()

,Length,vowels,consonants,stopwords,punctuations,special_char,tokens(whitespace),tokens(words),score_readability,label
0,511,167,278,0,1,1,66,66,48.04,0
1,724,239,389,0,0,0,95,95,62.23,0
2,1414,465,778,0,4,4,168,168,102.21,0
3,1514,505,822,0,3,3,185,185,109.62,0
4,1418,487,758,0,3,3,171,171,103.13,0


In [16]:
df.to_csv('data_description.csv', index=False)

## Convert RTF to TXT

In [29]:
from striprtf.striprtf import rtf_to_text

def convert_rtf_to_txt(directory):
    files = os.listdir(directory)

    for f in files:
        if os.path.isfile(os.path.join(directory, f)):
            filename, extension = os.path.splitext(f)

        if extension.lower() == ".rtf":
            rtf_file = open(os.path.join(directory, f), "r")
            rtf_content = rtf_file.read()
            text = rtf_to_text(rtf_content)
            rtf_file.close()

            new_name = f"{filename}.txt"
            txt_file = open(os.path.join(directory, new_name), "w")
            txt_file.write(text)
            txt_file.close()

            os.remove(os.path.join(directory, f))


for sub in subdirectories:
  convert_rtf_to_txt(sub)